### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv("mykeys.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f'{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&api-key={nyt_api_key}'


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for i in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url_page = query_url + f'&page={i}'

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_page).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews_list.append(reviews["response"]["docs"])
        
        # Print the page that was just retrieved
        print(f"Checked page {i}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"page {i} had no responses")


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [9]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0][:5],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2018/01/18/movies/kangaroo-a-love-hate-story-review.html",
        "snippet": "The documentary looks at the mass killings of kangaroos for pet-food companies, leather processors and ranchers in Australia.",
        "source": "The New York Times",
        "headline": {
            "main": "Review: \u2018Kangaroo: A Love-Hate Story\u2019 Exposes a Wildlife Massacre",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Kangaroo: A Love-Hate Story",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "creative_works",
                "value": "Kangaroo: A Love-Hate Story (Movie)",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "subject",
                "value": "Kangaroos",
                "rank": 2,
                "major": "N"
            }

In [37]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df= pd.json_normalize([item for sublist in reviews_list for item in sublist])
reviews_df

# alternate method
# flatten = []
# for sublist in reviews_list:
#     for item in sublist:
#         flatten.append(item)
# reviews_df = pd.DataFrame(pd.json_normalize(flatten))
# reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,"[{'name': 'creative_works', 'value': 'Kangaroo...",2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
1,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None
2,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-05-21T11:00:01+0000,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-11-05T19:01:02+0000,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
4,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/27/movies/ocea...,In a 1993 gem from this revered Japanese anima...,The New York Times,"[{'name': 'creative_works', 'value': 'Ocean Wa...",2016-12-27T21:08:58+0000,263,"Review: ‘Ocean Waves,’ a Tale of Young Love, G...",None,None,"Teenage Romance, Flashbacks and Style",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
196,https://www.nytimes.com/2022/05/26/movies/dinn...,A rage-fueled rock singer on the run from the ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-05-26T11:00:04+0000,254,‘Dinner in America’ Review: A Punk-Rock Love S...,None,None,Dinner in America,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
197,https://www.nytimes.com/2015/04/10/movies/revi...,"In this film, an orphaned girl’s friendship wi...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2015-04-09T23:37:10+0000,491,"Review: In ‘The Harvest,’ Love Is Wielded With...",None,None,"Love, Wielded With a Chill",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
198,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None


In [38]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df['title']  = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,"[{'name': 'creative_works', 'value': 'Kangaroo...",2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Kangaroo: A Love-Hate Story’ Exposes a Wildlif...
1,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerad
2,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-05-21T11:00:01+0000,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Tomorrow Man
3,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-11-05T19:01:02+0000,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Marriage Story
4,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Bones and All
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/27/movies/ocea...,In a 1993 gem from this revered Japanese anima...,The New York Times,"[{'name': 'creative_works', 'value': 'Ocean Wa...",2016-12-27T21:08:58+0000,263,"Review: ‘Ocean Waves,’ a Tale of Young Love, G...",None,None,"Teenage Romance, Flashbacks and Style",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Ocean Waves,’ a Tale of Young Love, Ghibli Styl"
196,https://www.nytimes.com/2022/05/26/movies/dinn...,A rage-fueled rock singer on the run from the ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-05-26T11:00:04+0000,254,‘Dinner in America’ Review: A Punk-Rock Love S...,None,None,Dinner in America,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Dinner in America
197,https://www.nytimes.com/2015/04/10/movies/revi...,"In this film, an orphaned girl’s friendship wi...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2015-04-09T23:37:10+0000,491,"Review: In ‘The Harvest,’ Love Is Wielded With...",None,None,"Love, Wielded With a Chill",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"The Harvest,’ Love Is Wielded With a Chil"
198,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The 

In [39]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,The New York Times,creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,263,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Kangaroo: A Love-Hate Story’ Exposes a Wildlif...
1,https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",The New York Times,"subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,272,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerad
2,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,The New York Times,subject: Movies;creative_works: The Tomorrow M...,2019-05-21T11:00:01+0000,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Tomorrow Man
3,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,The New York Times,"subject: Movies;persons: Baumbach, Noah;creati...",2019-11-05T19:01:02+0000,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Marriage Story
4,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,The New York Times,subject: Movies;creative_works: Bones and All ...,2022-11-17T12:00:06+0000,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Bones and All
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2016/12/27/movies/ocea...,In a 1993 gem from this revered Japanese anima...,The New York Times,creative_works: Ocean Waves (Movie);subject: A...,2016-12-27T21:08:58+0000,263,"Review: ‘Ocean Waves,’ a Tale of Young Love, G...",None,None,"Teenage Romance, Flashbacks and Style",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Ocean Waves,’ a Tale of Young Love, Ghibli Styl"
196,https://www.nytimes.com/2022/05/26/movies/dinn...,A rage-fueled rock singer on the run from the ...,The New York Times,"subject: Movies;persons: Rehmeier, Adam;creati...",2022-05-26T11:00:04+0000,254,‘Dinner in America’ Review: A Punk-Rock Love S...,None,None,Dinner in America,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Dinner in America
197,https://www.nytimes.com/2015/04/10/movies/revi...,"In this film, an orphaned girl’s friendship wi...",The New York Times,"subject: Movies;persons: McNaughton, John;pers...",2015-04-09T23:37:10+0000,491,"Review: In ‘The Harvest,’ Love Is Wielded With...",None,None,"Love, Wielded With a Chill",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"The Harvest,’ Love Is Wielded With a Chil"
198,https://www.nytimes.com/2022/11/17/movies/the-...,Elegance Bratton’s autobiographical first feat...,The New York Times,subject: Movies;creative_works: The Inspection...,2022-11-17T12:00:07+0000,836,"‘The Inspection’ Review: Boot Camp, a Love Story",critic’s pick,None,"A Few Good Men, Some With Secrets",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,The Inspection

In [40]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()
titles


['Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacr',
 'Love’s Eternal Masquerad',
 'The Tomorrow Man',
 'Marriage Story',
 'Bones and All',
 'Love, Antosha',
 'You Can Live Forever',
 'Prem Ratan Dhan Payo,’ a Bollywood Tale of a Prince-and-Plebe Doubl',
 'A Promoter Finds a New Cause: Go',
 'Now They’ll Love Me, a Twin Sister Scheme',
 'They’ll Love Me When I’m Dead’ Documents Orson Welles’s Last Fil',
 'A Journal for Jordan',
 'A United Kingdom’ With Love That Tested Racial Toleranc',
 'Sophie and the Rising Sun’ Has Forbidden Love in a Foreboding Tim',
 'Passengers’ Trapped on a Spaceship Find Love Amid Despai',
 'Undine',
 'Waiting for Bojangles',
 'Love and Unhappiness, in Soft Shades of Gra',
 'Lost in Paris',
 'Lost and Love’ (‘Shi Gu’) Stars Andy Lau as a Man Searching for His Lost So',
 'From This Day Forward’ Attests to Love’s Adaptabilit',
 'How He Fell in Love,’ an Extramarital Bond Unfold',
 'Young Love, Interrupted by a Nuclear Bom',
 'Love the Coopers,’ a Dysfunct

### Access The Movie Database API

In [41]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [42]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title_text in titles:
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0 :
        time.sleep(1)

    # Add 1 to the request counter
    request_counter = +1
    
    # Perform a "GET" request for The Movie Database
    url_string = url+title_text+tmdb_key_string
    response = requests.get(url_string).json()
    #print(json.dumps(response,indent=4))
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try: 
        # Get movie id
        movie_id = response["results"][0]["id"]
        #print (movie_id)

        # Make a request for a the full movie details
        url_id = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie_details = requests.get(url_id).json()
        #print (json.dumps(movie_details,indent=4))
        # Extract the genre names into a list
        genre_names = [movie_details["genres"][x]["name"] for x in range(len(movie_details["genres"]))]
        #print(genre_names)

        # Extract the spoken_languages' English name into a list
        spoken_languages = [movie_details["spoken_languages"][x]["name"] for x in range(len(movie_details["spoken_languages"]))]
        #print(spoken_languages)
        # Extract the production_countries' name into a list
        production_countries = [movie_details["production_countries"][x]["name"] for x in range(len(movie_details["production_countries"]))]
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {
            'title' : title_text,
            'original_title' : movie_details['original_title'],
            'budget' : movie_details['budget'],
            'original_language' : movie_details['original_language'],
            'homepage' : movie_details['homepage'],
            'overview' : movie_details['overview'],
            'popularity' : movie_details['popularity'],
            'runtime' : movie_details['runtime'],
            'revenue' : movie_details['revenue'],
            'release_date' : movie_details['release_date'],
            'vote_average' : movie_details['vote_average'],
            'vote_count' : movie_details['vote_count'],
            'genres' : genre_names,
            'spoken_languages' : spoken_languages, 
            'production_countries' : production_countries
        }
        
        tmdb_movies_list.append(movie_info)
        print(f"Found: {title_text}")

        # Print out the title that was found
    except:
        print(f"Not found: {title_text}")

Not found: Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacr
Not found: Love’s Eternal Masquerad
Found: The Tomorrow Man
Found: Marriage Story
Found: Bones and All
Found: Love, Antosha
Found: You Can Live Forever
Not found: Prem Ratan Dhan Payo,’ a Bollywood Tale of a Prince-and-Plebe Doubl
Not found: A Promoter Finds a New Cause: Go
Not found: Now They’ll Love Me, a Twin Sister Scheme
Not found: They’ll Love Me When I’m Dead’ Documents Orson Welles’s Last Fil
Found: A Journal for Jordan
Not found: A United Kingdom’ With Love That Tested Racial Toleranc
Not found: Sophie and the Rising Sun’ Has Forbidden Love in a Foreboding Tim
Not found: Passengers’ Trapped on a Spaceship Find Love Amid Despai
Found: Undine
Found: Waiting for Bojangles
Not found: Love and Unhappiness, in Soft Shades of Gra
Found: Lost in Paris
Not found: Lost and Love’ (‘Shi Gu’) Stars Andy Lau as a Man Searching for His Lost So
Not found: From This Day Forward’ Attests to Love’s Adaptabilit
Not found: How He F

In [43]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5],indent=4))


[
    {
        "title": "The Tomorrow Man",
        "original_title": "The Tomorrow Man",
        "budget": 0,
        "original_language": "en",
        "homepage": "https://bleeckerstreetmedia.com/thetomorrowman",
        "overview": "Ed Hemsler spends his life preparing for a disaster that may never come. Ronnie Meisner spends her life shopping for things she may never use. In a small town somewhere in America, these two people will try to find love while trying not to get lost in each other\u2019s stuff.",
        "popularity": 10.239,
        "runtime": 94,
        "revenue": 0,
        "release_date": "2019-05-22",
        "vote_average": 5.746,
        "vote_count": 59,
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Marriage Story",
        "original_title": "Marriage Story",
 

In [44]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_list_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Tomorrow Man,The Tomorrow Man,0,en,https://bleeckerstreetmedia.com/thetomorrowman,Ed Hemsler spends his life preparing for a dis...,10.239,94,0,2019-05-22,5.746,59,"[Drama, Romance]",[English],[United States of America]
1,Marriage Story,Marriage Story,18000000,en,https://www.marriagestorymovie.com,A stage director and an actress struggle throu...,37.504,137,2300000,2019-09-28,7.744,6767,[Drama],[English],[United Kingdom]
2,Bones and All,Bones and All,16000000,it,https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",49.707,131,15234907,2022-11-18,7.070,1257,"[Drama, Horror, Romance]",[English],"[Italy, United States of America]"
3,"Love, Antosha","Love, Antosha",0,en,https://antonyelchindoc.com/,"From a prolific career in film and television,...",6.283,92,0,2019-08-02,7.200,50,[Documentary],"[English, Pусский]",[United States of America]
4,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.235,96,15055,2023-03-24,6.600,37,"[Drama, Romance]","[English, Français]","[Canada, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,In From the Side,In from the Side,0,en,http://www.infromthesidemovie.com,"Mark, a new and inexperienced rugby club membe...",17.990,134,52885,2022-09-16,6.800,62,"[Drama, Romance]",[English],[United Kingdom]
99,Hot Air,Hot Air,0,en,,A right-wing talk show host's life takes a sud...,7.378,99,0,2018-04-10,5.100,18,[Drama],[English],[United States of America]
100,"Casanova, Last Love",Dernier amour,0,fr,https://lesfilmsdufleuve.be/en/movies/dernier-...,"Year 1763. Forced into exile, the famous liber...",14.980,99,0,2019-03-20,4.700,34,"[Drama, Romance]","[English, Français, Deutsch, Italiano]","[Belgium, France, United States of America]"
101,Dinner in America,Dinner in America,0,en,https://www.dinnerinamerica.com,An on-the-lam punk rocker and a young woman ob...,11.767,106,0,2022-05-27,7.384,56,"[Romance, Comedy, Music]",[English],[United States of America]


### Merge and Clean the Data for Export

In [49]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_movies_list_df,reviews_df, on='title',how='inner')
merged_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Tomorrow Man,The Tomorrow Man,0,en,https://bleeckerstreetmedia.com/thetomorrowman,Ed Hemsler spends his life preparing for a dis...,10.239,94,0,2019-05-22,...,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,Marriage Story,Marriage Story,18000000,en,https://www.marriagestorymovie.com,A stage director and an actress struggle throu...,37.504,137,2300000,2019-09-28,...,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
2,Bones and All,Bones and All,16000000,it,https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",49.707,131,15234907,2022-11-18,...,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
3,"Love, Antosha","Love, Antosha",0,en,https://antonyelchindoc.com/,"From a prolific career in film and television,...",6.283,92,0,2019-08-02,...,"‘Love, Antosha’ Review: A Heartbreaking Look a...",Critic’s Pick,None,"Love, Antosha",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.235,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,In From the Side,In from the Side,0,en,http://www.infromthesidemovie.com,"Mark, a new and inexperienced rugby club membe...",17.990,134,52885,2022-09-16,...,‘In From the Side’ Review: Love and Rugby Play...,None,None,In From the Side,None,None,None,By Kyle Turner,"[{'firstname': 'Kyle', 'middlename': None, 'la...",None
99,Hot Air,Hot Air,0,en,,A right-wing talk show host's life takes a sud...,7.378,99,0,2018-04-10,...,‘Hot Air’ Review: A Right-Wing Radio Host Lear...,None,None,Hot Air,None,None,None,By Bilge Ebiri,"[{'firstname': 'Bilge', 'middlename': None, 'l...",None
100,"Casanova, Last Love",Dernier amour,0,fr,https://lesfilmsdufleuve.be/en/movies/dernier-...,"Year 1763. Forced into exile, the famous liber...",14.980,99,0,2019-03-20,...,"‘Casanova, Last Love’ Review: Reappraising a P...",None,None,"Casanova, Last Love",None,None,None,By Beatrice Loayza,"[{'firstname': 'Beatrice', 'middlename': None,...",None
101,Dinner in America,Dinner in America,0,en,https://www.dinnerinamerica.com,An on-the-lam punk rocker and a young woman ob...,11.767,106,0,2022-05-27,...,‘Dinner in America’ Review: A Punk-Rock Love S...,None,None,Dinner in America,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None


In [57]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
character_to_remove = ['[',']']

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for line in merged_df[column]:
        merged_df[column] = merged_df[column].str.replace(r"\[|\]|\u2018|\u2019|'", '', regex=True)

# Display the fixed DataFrame
merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Tomorrow Man,The Tomorrow Man,0,en,https://bleeckerstreetmedia.com/thetomorrowman,Ed Hemsler spends his life preparing for a dis...,10.239,94,0,2019-05-22,...,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,None
1,Marriage Story,Marriage Story,18000000,en,https://www.marriagestorymovie.com,A stage director and an actress struggle throu...,37.504,137,2300000,2019-09-28,...,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,None
2,Bones and All,Bones and All,16000000,it,https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",49.707,131,15234907,2022-11-18,...,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,None
3,"Love, Antosha","Love, Antosha",0,en,https://antonyelchindoc.com/,"From a prolific career in film and television,...",6.283,92,0,2019-08-02,...,319,"‘Love, Antosha’ Review: A Heartbreaking Look a...",Critic’s Pick,None,"Love, Antosha",None,None,None,By Glenn Kenny,None
4,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.235,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None


In [51]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)

In [52]:
# Delete duplicate rows and reset index
cleaned_merged_df = merged_df.drop_duplicates().reset_index(drop=True)
cleaned_merged_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Tomorrow Man,The Tomorrow Man,0,en,https://bleeckerstreetmedia.com/thetomorrowman,Ed Hemsler spends his life preparing for a dis...,10.239,94,0,2019-05-22,...,280,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,None
1,Marriage Story,Marriage Story,18000000,en,https://www.marriagestorymovie.com,A stage director and an actress struggle throu...,37.504,137,2300000,2019-09-28,...,1244,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,None
2,Bones and All,Bones and All,16000000,it,https://www.bonesandallfilm.net/,"Abandoned by her father, a young woman embarks...",49.707,131,15234907,2022-11-18,...,710,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,None
3,"Love, Antosha","Love, Antosha",0,en,https://antonyelchindoc.com/,"From a prolific career in film and television,...",6.283,92,0,2019-08-02,...,319,"‘Love, Antosha’ Review: A Heartbreaking Look a...",Critic’s Pick,None,"Love, Antosha",None,None,None,By Glenn Kenny,None
4,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.235,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None


In [54]:
# Export data to CSV without the index
cleaned_merged_df.to_csv("output\cleaned_data.csv",index=False)